# Plot paired and unpaired synapses

This notebook assumes that all of the studies are orginized into a standard Python data structure. All of the plots are done using plot.ly, so you should look at the appropriate documentation for that if you want to change them.

The variable `study_list` constains a list of studies, where each study is represented as a pyton dictionary. The keys in the dictionary currently include the study name, the subject id, the type of study (aversion, control), and if the fish learned or not. 

The python command `list(study_list[0])` will return a list of the current values that are available for a study.

The study dictionary may also contain a set of calculated synapse pairings.  Each pair is represented as a python dictionary, whose key is the radius with which the pairing was determined, and whose value is a PANDA, with the synapse positions of the pair. Each PANDA has included in it a set of useful values from the study, including the study id, the type of study, the radius for the pair, etc.  Pairs are also transformed using the alignment points to put them into a common coordinate space.  The sets of pairs provided are:

* PairedBefore
* PairedAfter
* UnpairedBefore
* UnpairedAfter
* AlignedPairedBefore
* AlignedPairedAfter
* AlignedUnpairedBefore
* AlignedUnpairedAfter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Load up all of the libraries we need

import synapse_plot_utils as sp
import synapse_plot_config as plot_config
from synapse_plot_config import plot_synapses, trace_color, position_layout, step_buttons, studytypes

import pandas as pd
import numpy as np
import xarray as xr
import plotly as py
import plotly.graph_objs as go
import matplotlib
from matplotlib import cm

import colorlover as cl

# configure plot.ly to run in a notebook.
py.offline.init_notebook_mode(connected=True)

<H2> Useful Global Variables

In [ ]:
syn_pair_radii = (4, 300.0,) # maximum search radii
syn_dx_core_ratio = None  # turn off 4D nearest-neighbor
syn_core_max_ratio = None # turn off intensity ratio threshold

# Load Data

You have two choices, you can reload data from a previous notebook run, or regenerate all of the pairs with data from the catalog. If there have been new studies added, or any data has changed, you must regenerate all the pairs. In general, you should probably only use the save and 

<h3> Restore data from previous run

In [ ]:
dumpfile = 'pairs-dump.pkl'
studyset = sp.restore_studies(dumpfile)
studylist = studyset['Studies']
studyid = studyset['StudyID']
study_types = studytypes(studylist)

<H3> Get Study Pairs from Catalog

We are going to pull the synapse data down directly from the syapse server.  In order to do this we have to make a connection to the server at synapse.isrd.isi.edu You need to make sure you have logged in using the DERIVA login agent first

In [ ]:
dumpfile = 'pairs-dump.pkl'
studyid = 'TYR'

studyset = sp.get_studies(studyid)

for k,v in sp.group_studies(studyset['Studies'], group='Type').items():
    print('{0} {1}'.format(k, len(v)))

studyset['Studies'] = sp.compute_pairs(studyset['Studies'], syn_pair_radii)
studylist = studyset['Studies']
study_types = studytypes(studylist)

dumpfile = 'pairs-dump.pkl'

sp.dump_studies(studyset, dumpfile)


<H1> Compute some basic Statistics

In [ ]:
# Set what radius/studies you want to have....

cols = ['ID', 'Type', 'Radius', 'Unpaired Before', 'Unpaired After','Paired',]
c1, c2, c3, c4, c5, c6 = [],[],[],[],[],[]

syn_pair_radii = list(studylist[0]['UnpairedBefore'])

for s in studylist:
    for r in syn_pair_radii:
        c1.append(s['Study'])
        c2.append(s['Type'])
        c3.append(r)
        c4.append(len(s['UnpairedBefore'][r]['Data'].index))
        c5.append(len(s['UnpairedAfter'][r]['Data'].index))
        c6.append(len(s['PairedBefore'][r]['Data'].index))
    
trace = go.Table(
    header=dict(values=cols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[c1, c2, c3, c4, c5, c6],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

data = [trace] 
py.offline.iplot(data, filename = 'statistics_table')

<H1> Plot histograms of pair distance for all of the studies

In [ ]:
pair_distance = { k : pd.Series() for k in study_types}

rmax = max(studylist[0]['PairedAfter'])

for s in studylist:
    # Get the pairing for the maximum radius...
    before = s['PairedBefore'][rmax]['Data']
    after = s['PairedAfter'][rmax]['Data']
    t = s['Type']
    pair_distance[t] = pair_distance[t].append(pd.Series(np.sqrt((before['x'] - after['x'])**2 +
                        (before['y'] - after['y'])**2 +
                        (before['z'] - after['z'])**2)))

data = []
for i in study_types:
     data.append(go.Histogram(x=pair_distance[i], name='{0} Radias {1}'.format(i.capitalize(), rmax)))

hist_layout = go.Layout(
    title='Pair Distance Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=data, layout=hist_layout)

py.offline.iplot(hist_fig, filename='pair-distance')

<h1> Histogram of pair distances - past a certain pairing threshold 

In [ ]:
thres_pair=20.0 # Threshold for the minimum value to display on the histogram

hist_data_l_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))
hist_data_nl_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1))

# Normalized
hist_data_l_N_thres = go.Histogram(x=pair_distance['learner'], name='Learners Radius {0}'.format(r),
                          opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")
hist_data_nl_N_thres = go.Histogram(x=pair_distance['nonlearner'], name='Nonlearners Radius {0}'.format(r),
                           opacity=0.75,xbins=dict(start=thres_pair,end=90.0,size=1),histnorm="probability")

hist_layout_Norm = go.Layout(
    title='Pair Distance Histogram Radius Normalized {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig_thres = go.Figure(data=[hist_data_l_thres, hist_data_nl_thres], layout=hist_layout)
hist_fig_Norm_thres = go.Figure(data=[hist_data_l_N_thres, hist_data_nl_N_thres], layout=hist_layout_Norm)

py.offline.iplot(hist_fig_thres, filename='pair-distance_thresholded')
py.offline.iplot(hist_fig_Norm_thres, filename='pair-distance_Norm_thresholded')

<h1> Histogram of intensity ratios

In [ ]:
r  = min(studylist[0]['UnpairedBefore'])

core_ratio = { k : pd.Series() for k in study_types}
core = { k : pd.Series() for k in study_types}

for s in studylist:
    before = s['PairedBefore'][r]['Data']
    after = s['PairedAfter'][r]['Data']
    t = s['Type']
    core_ratio[t] = core_ratio[t].append(pd.Series(before['core'] / after['core']))
    core[t] = core_ratio[t].append(pd.Series(before['core']))
    core[t] = core_ratio[t].append(pd.Series(after['core']))

data = []
for i in study_types:
     data.append(go.Histogram(x=core_ratio[i], name='{0} Radias {1}'.format(i.capitalize(), r)))
    
hist_layout = go.Layout(
    title='Pair Ratio Histogram Radius {0}'.format(r),
    barmode='stack',
    xaxis=dict(
         title='Distance'
    ),
    yaxis=dict(
         title='Count'
        ),
    bargap=0.2,
     bargroupgap=0.1
    )

hist_fig = go.Figure(data=data, layout=hist_layout)

py.offline.iplot(hist_fig, filename='core-intensity')

<H2> Basic statistics for a study set

In [ ]:
c1,c2,c3,c4, c5 = [], [], [], [], []
synapses = sp.aggregate_studies(studylist)[0]

for k,v in sp.group_studies(studylist, group='Type').items():
    c1.append(k)
    c2.append(len(v))
    c3.append(synapses[k]['UnpairedBefore'].shape[0])
    c4.append(synapses[k]['UnpairedAfter'].shape[0])
    c5.append(synapses[k]['PairedBefore'].shape[0])
table = go.Table(
    header=dict(values=['Type', 'Count', 'Before Unpaired', 'After Unpaired', 'Paired'],
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 4),
    cells=dict(values=[c1, c2, c3, c4, c5],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))

layout = go.Layout(
    title='Summary Statistics Study Set SYNAPSE:{0}'.format(studyid),
    )

fig = go.Figure(data=[table], layout=layout)

py.offline.iplot(fig, filename = 'statistics_table')

<h2> Dump out CSV Files....

In [ ]:
testDict = {cols[0]:c1,cols[1]:c2,cols[2]:c3,cols[3]:c4,cols[4]:c5,cols[5]:c6}
with open('SynapseDataTable.csv', 'w') as f:  # Just use 'w' mode in 3.x
   w = csv.DictWriter(f, cols)
   w.writeheader()
   w = csv.writer(f,cols,delimiter=',', quoting=csv.QUOTE_NONE)
   for i in range(0,len(c1)):
       w.writerow((testDict[cols[0]][i],testDict[cols[1]][i],
                   testDict[cols[2]][i],testDict[cols[3]][i],
                   testDict[cols[4]][i],testDict[cols[5]][i]))
       
# For every nth row (so that you only look at one radius element at a time)
RowForRad = 3 #look at every RowForRad-th row
filename = 'SynapseDataTable_'+str(RowForRad)+'Rows.csv'
with open(filename, 'w') as f:  # Just use 'w' mode in 3.x
   w = csv.DictWriter(f, cols)
   w.writeheader()
   w = csv.writer(f,cols,delimiter=',', quoting=csv.QUOTE_NONE)
   for i in range(0,len(c1),RowForRad):
       w.writerow((testDict[cols[0]][i],testDict[cols[1]][i],
                   testDict[cols[2]][i],testDict[cols[3]][i],
                   testDict[cols[4]][i],testDict[cols[5]][i]))

<H1> Plot sets of synapses...

In [ ]:
# Get all radii for a specific study

#s_list = [s for s in paired_studies if s['Study'] in s_names]
# Pick which traces you would like displayed

tracelist =['AlignedUnpairedBefore','AlignedUnpairedAfter',
            'AlignedPairedBefore',
            'AlignmentPts']

print('Plotting {0} studies'.format(len(studylist)))

for k,v in sp.group_studies(studylist, group='Type').items():
    print('{0} {1}'.format(len(v), k))

#Pick the slider mode...
plotmode = 'Study'
plotmode = 'Trace'
plotmode = 'Studyset'


fig = plot_synapses(studylist, tracelist, plotmode, centroid=False, skipall=True) 

plot_file_name = 'synapse-plot'
py.offline.iplot(fig, filename=plot_file_name)
    
#py.offline.plot(fig,  filename =plot_file_name, output_type='file')
    

<h2> Plot summary of centroids

In [ ]:
centroidlist = []
tracelist =['AlignedUnpairedBefore','AlignedUnpairedAfter','AlignedPairedBefore', 'AlignmentPts']

# Use the smallest available radius as the default if one is not provided.
radius =  min(studylist[0][tracelist[0]])

for t in tracelist: 
    if t != 'AlignmentPts':
        centroidlist.append(t + 'Centroid')
before_traces, after_traces, paired_traces = [], [], []

cnt_before = cnt_after = cnt_paired = 0
before_x = before_y = before_z = 0
after_x = after_y = after_z = 0
paired_x = paired_y = paired_z = 0
maxx,maxy,maxz,minx,miny,minz = -float('inf'),-float('inf'),-float('inf'),float('inf'),float('inf'),float('inf')
    
for t in centroidlist:
    x,y,z, text = [],[],[],[]
    for s in studylist:
        x.append(s[t][radius]['Data']['x'][0])
        y.append(s[t][radius]['Data']['y'][0])
        z.append(s[t][radius]['Data']['z'][0])
        text.append(s['Study'])
        
    maxx = max(x + [maxx])
    maxy = max(y + [maxy])
    maxz = max(z + [maxz])
    minx = min(x + [minx])
    miny = min(y + [miny])
    minz = min(z + [minz])
    
    centroid_trace = go.Scatter3d(
        x = x, y = y, z = z,
        mode='markers',
        name = '{0}'.format(t),
        text = text,
        marker=dict(
            size = 6, color = trace_color(t),
            line=dict(color='black', width=0.8))
        )

    if 'UnpairedBefore' in t:
        before_x = sum(x) + before_x
        before_y = sum(y) + before_y
        before_z = sum(z) + before_z
        cnt_before = cnt_before + len(x)
        before_traces.append(centroid_trace)
    elif 'UnpairedAfter' in t:
        after_x = sum(x) + after_x
        after_y = sum(y) + after_y
        after_z = sum(z) + after_z
        cnt_after = cnt_after + len(x)
        after_traces.append(centroid_trace)
    elif 'PairedBefore' in t:
        paired_x = sum(x) + paired_x
        paired_y = sum(y) + paired_y
        paired_z = sum(z) + paired_z
        cnt_paired = cnt_paired + len(x)
        paired_traces.append(centroid_trace)
        
summary_trace = go.Scatter3d(
    x = [ before_x / cnt_before ], y = [ before_y / cnt_before ], z = [ before_z / cnt_before ],
    mode = 'markers',
    name = 'Centroids Before',
    marker = dict(
        size = 8,
        symbol = 'square',
        color = ['orange']
     )
)
before_traces.append(summary_trace)

summary_trace = go.Scatter3d(
    x = [ after_x / cnt_after ],  y = [ after_y / cnt_after ], z = [ after_z / cnt_after ],
    mode = 'markers',
    name = 'Centroids After',
    marker = dict(
        size = 8,
        symbol = 'square',
        color = ['blue']
     )
)
after_traces.append(summary_trace)

summary_trace = go.Scatter3d(
    x = [ paired_x / cnt_paired ],  y = [ paired_y / cnt_paired ], z = [ paired_z / cnt_paired ],
    mode = 'markers',
    name = 'Centroids Paired',
    marker = dict(
        size = 8,
        symbol = 'square',
        color = ['green']
     )
)
paired_traces.append(summary_trace)

masks = {'study' : 
         { 'centroid' : 
          {
    'all' :     [True] * (len(before_traces) + len(after_traces) + len(paired_traces)),
    'before' :  [True] * len(before_traces) + [False] * (len(after_traces) + len(paired_traces)),
    'after' : [False] * len(before_traces) + [True] * len(after_traces) + [False] * len(paired_traces),
    'paired' : [False] * (len(before_traces) + len(after_traces)) + [True] * len(paired_traces),
          }
         }
    }

layout = position_layout(minx, miny, minz, maxx, maxy, maxz)
layout['title'] = 'Centroids'
layout['updatemenus'] = step_buttons('study', masks, study_types, step='centroid')

fig = dict(data=before_traces + after_traces + paired_traces, layout=layout)

plot_file_name = 'centroid-plot'

py.offline.iplot(fig, filename=plot_file_name)
    
#py.offline.plot(fig,  filename =plot_file_name, output_type='file')

In [ ]:
# Contour plots.....

plot_file_name = 'centroid-plot'
coord0 = 'x'
coord1 = 'z'

scl_b = cl.scales['9']['seq']['Blues']
colorscale_b = [ [ float(i)/float(len(scl_b)-1), scl_b[i] ] for i in range(len(scl_b)) ]
scl_o = cl.scales['9']['seq']['Oranges']
colorscale_o = [ [ float(i)/float(len(scl_o)-1), scl_o[i] ] for i in range(len(scl_o)) ]
scl_g = cl.scales['9']['seq']['Greens']
colorscale_g = [ [ float(i)/float(len(scl_g)-1), scl_g[i] ] for i in range(len(scl_g)) ]

colorscale_b[0] = [colorscale_b[0][0], 'white']
colorscale_o[0] = [colorscale_o[0][0], 'white']
colorscale_g[0] = [colorscale_o[0][0], 'white']

data = []
studyset = {}
masks = {'studyset': studyset}

# We have a trace for all/before/all and center of mass for each study type, 
trace_cnt = (len(study_types)) * 6
for i in study_types:
    studyset[i] = {j: [False] * trace_cnt for j in ['all', 'before', 'after', 'paired']}

synapses, syn_max, syn_min = sp.aggregate_studies(studylist)


data = []
for i in study_types:
    # Set mask for this trace 

    studyset[i]['all'][len(data)] = True
    studyset[i]['before'][len(data)] = True

    data.append(go.Histogram2dcontour(
        histnorm='density',
        visible=False,
        x = synapses[i]['UnpairedBefore'][coord0], y = synapses[i]['UnpairedBefore'][coord1],
        name='before density', 
        colorscale=colorscale_o, reversescale=False, showscale=False, opacity=0.5)
    )
    
    cm0 = synapses[i]['UnpairedBefore'][coord0].sum()/synapses[i]['UnpairedBefore'].shape[0]
    cm1 = synapses[i]['UnpairedBefore'][coord1].sum()/synapses[i]['UnpairedBefore'].shape[0] 
    studyset[i]['all'][len(data)] = True
    studyset[i]['before'][len(data)] = True
    data.append(go.Scatter(
        visible=False,
        x=[cm0], y = [cm1],
        mode = 'markers', marker = dict(size = 10, color = 'orange', symbol = 'cross'))
    )
    
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['after'][len(data)] = True
    data.append(go.Histogram2dcontour(
        histnorm='density',
            visible=False,
            x = synapses[i]['UnpairedAfter'][coord0], y = synapses[i]['UnpairedAfter'][coord1],
            name='after density', #ncontours=5,
         #   xaxis='after_x', yaxis='after_y',
                        colorscale=colorscale_b, reversescale=False, showscale=False, opacity=0.5) 
    )
    
    cm0 = synapses[i]['UnpairedAfter'][coord0].sum()/synapses[i]['UnpairedAfter'].shape[0]
    cm1 = synapses[i]['UnpairedAfter'][coord1].sum()/synapses[i]['UnpairedAfter'].shape[0]
    studyset[i]['all'][len(data)] = True
    studyset[i]['after'][len(data)] = True
    data.append(go.Scatter(
        visible=False,
        x=[cm0], y = [cm1],
        mode = 'markers', marker = dict(size = 10, color = 'blue', symbol = 'cross'))
    )

    studyset[i]['all'][len(data)] = True
    studyset[i]['paired'][len(data)] = True
    data.append(go.Histogram2dcontour(
        histnorm='density',
            visible=False,
            x = synapses[i]['UnpairedAfter'][coord0], y = synapses[i]['UnpairedAfter'][coord1],
            name='paired density', #ncontours=5,
         #   xaxis='after_x', yaxis='after_y',
                        colorscale=colorscale_g, reversescale=False, showscale=False, opacity=0.5)
    )
                
    cm0 = synapses[i]['PairedBefore'][coord0].sum()/synapses[i]['PairedBefore'].shape[0]
    cm1 = synapses[i]['PairedBefore'][coord1].sum() /synapses[i]['PairedBefore'].shape[0]
    studyset[i]['all'][len(data)] = True
    studyset[i]['paired'][len(data)] = True
    data.append(go.Scatter(
        visible=False,
        x=[cm0], y = [cm1],
        mode = 'markers', marker = dict(size = 10, color = 'green', symbol = 'cross'))
    )
    
    
layout = go.Layout(
    showlegend=False,
    autosize=False,
    title='Synapse Density: Learner All',
  #  width=600,
    height=800,
    xaxis=dict(
        domain=[0, 1],
        range=[syn_min[0],syn_max[0]],
        title='X',
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        domain=[0, .7],
        title='Z',
        range=[syn_min[2],syn_max[2]],
        scaleanchor='x',
        showgrid=False,
        zeroline=False
    ),
    hovermode='closest',
)

for i,v in enumerate(masks['studyset']['learner']['all']):
    data[i]['visible'] = v
layout['updatemenus'] = step_buttons('studyset', masks, study_types, showlegend=False, title='Synapse Density: ',
                                     skipall=True)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig, filename=plot_file_name)

In [ ]:
plot_file_name = 'synapse-density'

scl_b = cl.scales['9']['seq']['Blues']
colorscale_b = [ [ float(i)/float(len(scl_b)-1), scl_b[i] ] for i in range(len(scl_b)) ]
scl_o = cl.scales['9']['seq']['Oranges']
colorscale_o = [ [ float(i)/float(len(scl_o)-1), scl_o[i] ] for i in range(len(scl_o)) ]
scl_g = cl.scales['9']['seq']['Greens']
colorscale_g = [ [ float(i)/float(len(scl_g)-1), scl_g[i] ] for i in range(len(scl_g)) ]

colorscale_b[0] = [colorscale_b[0][0], 'white']
colorscale_o[0] = [colorscale_o[0][0], 'white']
colorscale_g[0] = [colorscale_o[0][0], 'white']

data = []
studyset = {}
masks = {'studyset': studyset}

# We have a trace for all/before/all and center of mass for each study type, 
trace_cnt = (len(study_types)) * 6
for i in study_types:
    studyset[i] = {j: [False] * trace_cnt for j in ['all', 'before', 'after', 'paired']}

binned_synapses = sp.bin_synapses(studylist, nbins = 10)
# Compute the count
#for t, counts in binned_synapses.items():
#    counts['PairedBefore'] = xr.ufuncs.fabs(counts['Before'] - counts['After'])

synapse_density = sp.synapse_density(binned_synapses)

for i in study_types:
    studyset[i]['all'][len(data)] = True
    studyset[i]['before'][len(data)] = True

    density = synapse_density[i]['UnpairedBefore']
    data.append(go.Heatmap(
        visible=False,
        z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
        name='before density', 
        colorscale=colorscale_o, reversescale=False, showscale=False, opacity=0.5)
    )
      
    studyset[i]['all'][len(data)] = True
    studyset[i]['before'][len(data)] = True
    center_of_mass = density.attrs['center_of_mass']  
    data.append(go.Scatter(
        visible=False,
        x=[center_of_mass[0]], y = [center_of_mass[1]],
        mode = 'markers', marker = dict(size = 10, color = 'orange', symbol = 'cross'))
    )
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['after'][len(data)] = True
    density = synapse_density[i]['UnpairedAfter']
    data.append(go.Heatmap(
            visible=False,
            z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
            name='after density', #ncontours=5,
         #   xaxis='after_x', yaxis='after_y',
                        colorscale=colorscale_b, reversescale=False, showscale=False, opacity=0.5)
        )
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['after'][len(data)] = True
    center_of_mass = density.attrs['center_of_mass'] 
    data.append(go.Scatter(
        visible=False,
        x=[center_of_mass[0]], y = [center_of_mass[1]],
        mode = 'markers', marker = dict(size = 10, color = 'blue', symbol = 'cross'))
    )
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['paired'][len(data)] = True
    density = synapse_density[i]['PairedBefore']
    data.append(go.Heatmap(
        visible=False,
            z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
            name='paired density',
        colorscale=colorscale_g, reversescale=False, showscale=False, opacity=0.5)
               )
 
    studyset[i]['all'][len(data)] = True
    studyset[i]['paired'][len(data)] = True
    center_of_mass = density.attrs['center_of_mass'] 
    data.append(go.Scatter(
        visible=False,
        x=[center_of_mass[0]], y = [center_of_mass[1]],
        mode = 'markers', marker = dict(size = 10, color = 'green', symbol = 'cross'))
    )
    
smin = synapse_density['learner'].attrs['min']
smax = synapse_density['learner'].attrs['max']
layout = go.Layout(
    showlegend=False,
    autosize=False,
    title='Synapse Density: Learner All',
  #  width=600,
    height=700,
    xaxis=dict(
        domain=[0, 1],
        range=[smin[0],smax[0]],
        title='X',
        showgrid=False,
        zeroline=False
    ),
    yaxis=dict(
        domain=[0, .7],
        title='Z',
        range=[smin[1],smax[1]],
        scaleanchor='x',
        showgrid=False,
        zeroline=False
    ),
    hovermode='closest',
)

for i,v in enumerate(masks['studyset']['learner']['all']):
    data[i]['visible'] = v
layout['updatemenus'] = step_buttons('studyset', masks, study_types, showlegend=False, title='Synapse Density: ',
                                     skipall=True)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig, filename=plot_file_name)

In [ ]:
# Psudo pairing.....

plot_file_name = 'synapse-density'

scl_b = cl.scales['9']['seq']['Blues']
colorscale_b = [ [ float(i)/float(len(scl_b)-1), scl_b[i] ] for i in range(len(scl_b)) ]
scl_o = cl.scales['9']['seq']['Oranges']
colorscale_o = [ [ float(i)/float(len(scl_o)-1), scl_o[i] ] for i in range(len(scl_o)) ]
scl_g = cl.scales['9']['seq']['Greens']
colorscale_g = [ [ float(i)/float(len(scl_g)-1), scl_g[i] ] for i in range(len(scl_g)) ]

colorscale_b[0] = [colorscale_b[0][0], 'white']
colorscale_o[0] = [colorscale_o[0][0], 'white']
colorscale_g[0] = [colorscale_o[0][0], 'white']

data = []
studyset = {}
masks = {'studyset': studyset}

# We have a trace for all/before/all and center of mass for each study type, 
trace_cnt = (len(study_types)) * 6
for i in study_types:
    studyset[i] = {j: [False] * trace_cnt for j in ['all', 'before', 'after', 'paired']}

binned_synapses = sp.bin_synapses(studylist, nbins = 10)
# Compute the count
for t, counts in binned_synapses.items():
    counts['Diff'] = xr.ufuncs.fabs(counts['Before'] - counts['After'])

synapse_density = sp.synapse_density(binned_synapses)

for i in study_types:
    studyset[i]['all'][len(data)] = True
    studyset[i]['before'][len(data)] = True

    density = synapse_density[i]['UnpairedBefore']
    data.append(go.Heatmap(
        visible=False,
        z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
        name='before density', 
        colorscale=colorscale_o, reversescale=False, showscale=False, opacity=0.5)
    )
      
    studyset[i]['all'][len(data)] = True
    studyset[i]['before'][len(data)] = True
    center_of_mass = density.attrs['center_of_mass']  
    data.append(go.Scatter(
        visible=False,
        x=[center_of_mass[0]], y = [center_of_mass[1]],
        mode = 'markers', marker = dict(size = 10, color = 'orange', symbol = 'cross'))
    )
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['after'][len(data)] = True
    density = synapse_density[i]['UnpairedAfter']
    data.append(go.Heatmap(
            visible=False,
            z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
            name='after density', #ncontours=5,
         #   xaxis='after_x', yaxis='after_y',
                        colorscale=colorscale_b, reversescale=False, showscale=False, opacity=0.5)
        )
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['after'][len(data)] = True
    center_of_mass = density.attrs['center_of_mass'] 
    data.append(go.Scatter(
        visible=False,
        x=[center_of_mass[0]], y = [center_of_mass[1]],
        mode = 'markers', marker = dict(size = 10, color = 'blue', symbol = 'cross'))
    )
    
    studyset[i]['all'][len(data)] = True
    studyset[i]['paired'][len(data)] = True
    density = synapse_density[i]['Diff']
    data.append(go.Heatmap(
        visible=False,
            z = density.data, x = density.coords['x'].data, y = density.coords['z'].data,
            name='paired density',
        colorscale=colorscale_g, reversescale=False, showscale=False, opacity=0.5)
               )
 
    studyset[i]['all'][len(data)] = True
    studyset[i]['paired'][len(data)] = True
    center_of_mass = density.attrs['center_of_mass'] 
    data.append(go.Scatter(
        visible=False, x=[center_of_mass[0]], y = [center_of_mass[1]],
        mode = 'markers', marker = dict(size = 10, color = 'green', symbol = 'cross'))
    )
    
smin = synapse_density['learner'].attrs['min']
smax = synapse_density['learner'].attrs['max']
layout = go.Layout(
    showlegend=False,
    autosize=False,
    title='Synapse Density: Learner All',
  #  width=600,
    height=700,
    xaxis=dict(
        domain=[0, 1], range=[smin[0],smax[0]], title='X',
        showgrid=False, zeroline=False
    ),
    yaxis=dict(
        domain=[0, .7], title='Z', range=[smin[1],smax[1]],
        scaleanchor='x', showgrid=False, zeroline=False
    ),
    hovermode='closest',
)

for i,v in enumerate(masks['studyset']['learner']['all']):
    data[i]['visible'] = v
layout['updatemenus'] = step_buttons('studyset', masks, study_types, showlegend=False, title='Synapse Density: ',
                                     skipall=True)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig, filename=plot_file_name)